In [ ]:
import fipy as fp
from matplotlib import pyplot as plt
import numpy as np

m = fp.Grid1D(nx=50,Lx=2)
v = 0.1
epsilon= 0.1
delta = 0.1
x = m.cellCenters[0]
xc=np.asarray(m.cellCenters)[0]


##Initial conditions
c = fp.CellVariable(name="c", mesh=m , value=1., hasOld=True)

rho = fp.CellVariable(name="rho", mesh=m , value=0., hasOld=True)

phi = fp.CellVariable(name="phi",mesh=m, hasOld=True)
phi.setValue(v*(x-1))

#Plot the initial values
plt.plot(xc, phi.value, label='$\phi$')
plt.plot(xc, c.value, label='$c$')
plt.plot(xc, rho.value, label='$\\rho$')
plt.xlabel('x')
plt.legend(loc=7)
plt.show()

cFace = c.faceValue
rhoFace = rho.faceValue
phiFace = phi.faceValue

#vi =Viewer((phi, c, rho),datamin=-1.1, datamax=1.1)

#Calculate cell distance
from fipy.tools import numerix
MA = numerix.MA

tmp = MA.repeat(m._faceCenters[..., numerix.NewAxis,:], 2, 1)
cellToFaceDistanceVectors = tmp - numerix.take(m._cellCenters, m.faceCellIDs, axis=1)

tmp = numerix.take(m._cellCenters, m.faceCellIDs, axis=1)
tmp = tmp[..., 1,:] - tmp[..., 0,:]
cellDistanceVectors = MA.filled(MA.where(MA.getmaskarray(tmp), cellToFaceDistanceVectors[:, 0], tmp))

mask = m.exteriorFaces
Gamma = fp.FaceVariable(mesh=m, value=epsilon*epsilon)
Gamma.setValue(0., where=mask)
dPf = fp.FaceVariable(mesh=m,
                   value=m._faceToCellDistanceRatio * cellDistanceVectors)
n = m.faceNormals
a = m.faceNormals
b = fp.FaceVariable(mesh=m, value=epsilon * delta, rank=0)
g = fp.FaceVariable(mesh=m, value=v, rank=0)
RobinCoeff = mask * epsilon * epsilon * n / (-dPf.dot(a) + b)

The Robin codition below is modified from your original code by $\verb+ (RobinCoeff*g).divergence+$ to $\verb+ (RobinCoeff*g*n).divergence+$ 

In [ ]:
##Equation
Drho = epsilon*rho
Dc   = epsilon*c

eq1 = (fp.TransientTerm(var=c) == fp.DiffusionTerm(coeff=epsilon, var=c) +fp.DiffusionTerm(coeff=Drho, var=phi))
eq2 = (fp.TransientTerm(var=rho) == fp.DiffusionTerm(coeff=epsilon, var=rho) +fp.DiffusionTerm(coeff=Dc, var=phi))
eq3 = (fp.DiffusionTerm(coeff=Gamma, var=phi) + (RobinCoeff * g*n).divergence
       - fp.ImplicitSourceTerm(coeff=(RobinCoeff).divergence, var=phi) + fp.ImplicitSourceTerm(coeff=1.0, var=rho))
eqns = eq1 & eq2 & eq3 

The piece $\verb+fp.ImplicitSourceTerm(coeff=(RobinCoeff).divergence, var=dphi)+$ was inserted for the Robin condition for dphi

In [ ]:
dc = fp.CellVariable(mesh=m, name=r"$\delta c$", hasOld=True)
drho = fp.CellVariable(mesh=m, name=r"$\delta rho$", hasOld=True)
dphi = fp.CellVariable(mesh=m, name=r"$\delta phi$", hasOld=True)

dcEq =((fp.TransientTerm(var=dc)==fp.DiffusionTerm(coeff=epsilon, var=dc)+fp.ConvectionTerm(coeff=epsilon*phi.faceGrad,var=drho)+fp.DiffusionTerm(coeff=epsilon*rho, var=dphi))+fp.ResidualTerm(equation=eq1))

drhoEq = ((fp.TransientTerm(var=drho)==fp.ConvectionTerm(coeff=epsilon*phi.faceGrad, var=dc)+fp.DiffusionTerm(coeff=epsilon,var=drho)+fp.DiffusionTerm(coeff=epsilon*c, var=dphi))+fp.ResidualTerm(equation=eq2))

dphiEq =((fp.ImplicitSourceTerm(coeff=1.0, var=drho)+fp.DiffusionTerm(coeff=Gamma, var=dphi)-fp.ImplicitSourceTerm(coeff=(RobinCoeff).divergence, var=dphi))+fp.ResidualTerm(equation=eq3)) 

dEq = dcEq & drhoEq & dphiEq

In [ ]:
dc.value = 2.
drho.value = 1.
dphi.value  = 1.

##Sweep
newton = []
c.updateOld()
rho.updateOld()
phi.updateOld()
dc.updateOld()
drho.updateOld()
dphi.updateOld()
for sweep in xrange(10):
    res = dEq.sweep(dt=10)
    c.value = c.value + dc.value
    rho.value = rho.value + drho.value
    phi.value = phi.value+drho.value
    newton.append([sweep, res, max(abs(dc)), max(abs(drho)), max(abs(dphi))])
    
newton = numerix.array(newton)
print(newton)

##Print
plt.semilogy(newton[...,0], newton[..., 1], label="newton")
plt.ylabel("residual")
plt.xlabel("sweep")
plt.legend()
plt.show()

print(phi.value)
#Plot the results
plt.plot(xc, phi.value, label='$\phi$ - soln')
plt.plot(xc, c.value, label='$c$-soln')
plt.plot(xc, rho.value, label='$\\rho$ - soln')
plt.xlabel('x')
plt.legend(loc=7)
plt.show()
